In [ ]:
import json
import typing
import requests
import bs4
from urllib.parse import urljoin
import pymongo


def parse_post(url, soup):
    author_tag = soup.find("div", attrs={"itemprop": "author"}) # находим где в супе про автора
    data = {
                    "title": soup.find("h1", attrs={"class": "blogpost-title"}).text,# находим заголовок поста
                    "url": url,
                    "img_source":urljoin(url, soup.find("img").attrs.get("src", "/")),
                    'data':soup.find("time").attrs.get("datetime", "/"),
                    "author_data": urljoin(url, author_tag.parent.attrs.get("href")), #находим адрес автора
                    "name": author_tag.text,
                    "comments":get_comment(soup.find('div',attrs={'class':"referrals-social-buttons-small-wrapper"})['data-minifiable-id']),
                    "tags_data": [{"url": urljoin(url, tag_a.attrs.get("href")), "name": tag_a.text} 
                    for tag_a in soup.find_all("a", attrs={"class": "small"})]
            }
    return data
    
    
def save(data):
    collection = db["GeekBrains"]
    collection.insert_one(data)  
    
def urlpage(url,soup):
    post_urls = set(
            urljoin(url, url_a.attrs.get("href"))
            for url_a in soup.find_all("a", attrs={"class": "post-item__title"})
            if url_a.attrs.get("href"))
    for post_url in post_urls:
            if post_url not in urlpages:
                urlpages.add(post_url)
    return post_urls

def get_soup(url):
    response = requests.get(url)#, *args, **kwargs)
    soup = bs4.BeautifulSoup(response.text, "lxml")
    return soup
        
def get_comment(post_id):
    api_path=f'https://geekbrains.ru/api/v2/comments?commentable_type=Post&commentable_id={post_id}&order=desc'
    response = requests.get(api_path)
    data = response.json()
    return data
    
    
start_url = "https://geekbrains.ru/posts"
urlpages = set()

mongo_url = "mongodb://localhost:27017"
client = pymongo.MongoClient(mongo_url)
db = client["gb_parse_03_04_21"]



urlpage(start_url,get_soup(start_url))

count =2
while True:
    url = start_url+(f'?page={count}')
    print(url)
    soup = get_soup(url)
    urlpage(url,soup)
    if soup.find_all("a", attrs={"class": "post-item__title"}):
        count +=1
    else:
        break

for url in urlpages:
    print(url)
    data = parse_post(url, get_soup(url))
    save(data)